<a href="https://colab.research.google.com/github/Rito43/Rito43/blob/main/M23CSA021_iris_datset_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to your file
file_path = '/content/drive/MyDrive/hymenoptera_data '

# Get the directory of the file
directory = os.path.dirname(file_path)

print("Directory of the file:", directory)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from skimage import io
import os

import os
from skimage import io
from torch.utils.data import Dataset

class CustomHymenopteraDataset(Dataset):
    def __init__(self, root_dir, subset='train', transform=None):
        self.root_dir = root_dir
        self.subset = subset
        self.transform = transform
        self.images, self.labels = self._load_images_and_labels()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = io.imread(img_path)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

    def _load_images_and_labels(self):
        images = []
        labels = []
        subset_dir = os.path.join(self.root_dir, self.subset)
        for subdir in os.listdir(subset_dir):
            subdir_path = os.path.join(subset_dir, subdir)
            if os.path.isdir(subdir_path):
                for filename in os.listdir(subdir_path):
                    file_path = os.path.join(subdir_path, filename)
                    if os.path.isfile(file_path) and not filename.startswith('.'):
                        images.append(file_path)
                        labels.append(subdir)  # Use the folder name as the label
        return images, labels


# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define the root directory where the images are stored
root_dir = '/content/drive/MyDrive/hymenoptera_data'

# Create CustomHymenopteraDataset instances for train and test datasets
train_dataset = CustomHymenopteraDataset(root_dir=root_dir, subset='train', transform=transform)
test_dataset = CustomHymenopteraDataset(root_dir=root_dir, subset='val', transform=transform)

# Create DataLoader instances for train and test datasets
train_batch_size = 64
test_batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.datasets import fetch_openml
import numpy as np
from skimage.color import rgb2lab, lab2rgb


# Define the CAE architecture
class CAELABTORGB(nn.Module):
    def __init__(self):
        super(CAELABTORGB, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize CAE model
model_lab_to_rgb = CAELABTORGB()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_lab_to_rgb.parameters(), lr=0.001)

# Train the CAE model
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0

    for images,_ in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        images_lab = rgb2lab(images.permute(0, 2, 3, 1))
        images_lab_tensor = torch.tensor(images_lab.transpose(0,3,1,2), dtype=torch.float32)

        # Forward pass
        outputs = model_lab_to_rgb(images_lab_tensor)

        # Compute the loss
        loss = criterion(outputs, images)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)


    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f},')


In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define evaluation function
def evaluate(model, dataloader):
    model.eval()
    with torch.no_grad():
        for images,_ in dataloader:
            images_lab = rgb2lab(images.permute(0, 2, 3, 1))
            images_lab_tensor = torch.tensor(images_lab.transpose(0,3,1,2), dtype=torch.float32)
            outputs_rgb_tensor = model(images_lab_tensor)
            outputs_rgb = outputs_rgb_tensor.permute(0, 2, 3, 1).numpy()
            visualize(images_lab, outputs_rgb)
            images_rgb = images.permute(0, 2, 3, 1).numpy()
            outputs_rgb_flat = outputs_rgb.reshape(-1, 3)  # Reshape to 2D array
            images_rgb_flat = images_rgb.reshape(-1, 3)    # Reshape to 2D array
            mse_value = mse(images_rgb_flat,outputs_rgb_flat)
            ssim_value = ssim(images,outputs_rgb, win_size=3, multichannel=True, data_range=1)
            psnr_value = psnr(images,outputs_rgb,data_range=1)
            print(f'MSE: {mse_value:.4f}, SSIM: {ssim_value:.4f}, PSNR: {psnr_value:.4f}')

# Visualize input and reconstructed images
def visualize(images, outputs):
    # Visualize the first few images and their reconstructions
    num_images = min(5, len(images))
    fig, axes = plt.subplots(2, num_images, figsize=(12, 4))
    for i in range(num_images):
        image = np.clip(lab2rgb(images[i]), 0, 1)
        axes[0, i].imshow(image)
        axes[0, i].set_title('Input')
        axes[0, i].axis('off')
        output = np.clip(outputs[i], 0, 1)
        axes[1, i].imshow(output)
        axes[1, i].set_title('Reconstructed')
        axes[1, i].axis('off')
    plt.show()


# Evaluate the model
evaluate(model_lab_to_rgb, test_dataloader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# 1. Extract features using the trained autoencoder
encoder = model_lab_to_rgb.encoder.eval()  # Set to evaluation mode

# 2. Define the MLP classifier
class LABToRGBMLPClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LABToRGBMLPClassifier, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

writer = SummaryWriter()


lab_to_rgb_mlp = LABToRGBMLPClassifier(input_size=100352, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lab_to_rgb_mlp.parameters(), lr=0.001)
label_to_index = {'ants': 0, 'bees': 1}

# 5. Train the MLP classifier
num_epochs = 10
for epoch in range(num_epochs):
    lab_to_rgb_mlp.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_dataloader:
        # Forward pass
        encoder_output = encoder(inputs)

        # Flatten the output
        encoder_output_flat = encoder_output.view(encoder_output.size(0), -1)

        outputs = lab_to_rgb_mlp(encoder_output_flat)
        labels = [label_to_index[label] for label in labels]
        labels = torch.tensor(labels)


        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track running loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
    # Log metrics using Tensorboard
    writer.add_scalar('Loss/train',epoch_loss , epoch)
    writer.add_scalar('Accuracy/train',epoch_accuracy , epoch)

    # Validate the model
    lab_to_rgb_mlp.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            labels = [label_to_index[label] for label in labels]
            labels = torch.tensor(labels)

             # Forward pass
            encoder_output = encoder(inputs)

             # Flatten the output
            encoder_output_flat = encoder_output.view(encoder_output.size(0), -1)

            outputs = lab_to_rgb_mlp(encoder_output_flat)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Log validation accuracy
    writer.add_scalar('Accuracy/val', 100 * correct / total, epoch)

# 6. Use Tensorboard for visualization
# Make sure to close the Tensorboard writer after training
writer.close()

'Part B'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.datasets import fetch_openml
import numpy as np


# Define the CAE architecture
class CAERGBTONEG(nn.Module):
    def __init__(self):
        super(CAERGBTONEG, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2)
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def rgb_to_negative(images):
    negative_images = 1 - images
    return negative_images

# Initialize CAE model
model_rgb_to_neg = CAERGBTONEG()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_rgb_to_neg.parameters(), lr=0.001)

# Train the CAE model
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0

    for images,_ in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        images_neg = rgb_to_negative(images)

        # Forward pass
        outputs = model_rgb_to_neg(images)

        # Compute the loss
        loss = criterion(outputs, images_neg)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)


    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f},')


In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define evaluation function
def evaluate_neg(model, dataloader):
    model.eval()
    with torch.no_grad():
        for images, _ in dataloader:
            outputs_neg = model(images)

            images = images.permute(0, 2, 3, 1).numpy()
            outputs_neg = outputs_neg.permute(0, 2, 3, 1).numpy()

            visualize_neg(images, outputs_neg)

            outputs_neg_flat = outputs_neg.reshape(-1, 3)  # Reshape to 2D array
            images_neg_flat = rgb_to_negative(images).reshape(-1, 3)  # Reshape to 2D array

            mse_value = mse(images_neg_flat,outputs_neg_flat)
            images_neg = rgb_to_negative(images)

            ssim_value = ssim(images, outputs_neg, win_size=3, data_range=1, multichannel=True)
            psnr_value = psnr(images, outputs_neg, data_range=1)

            print(f'MSE: {mse_value:.4f}, SSIM: {ssim_value:.4f}, PSNR: {psnr_value:.4f}')

# Visualize input and reconstructed images
def visualize_neg(images, outputs):
    # Visualize the first few images and their reconstructions
    num_images = min(5, len(images))
    fig, axes = plt.subplots(2, num_images, figsize=(12, 4))
    for i in range(num_images):
        image = np.clip(images[i], 0, 1)
        axes[0, i].imshow(image)
        axes[0, i].set_title('Input')
        axes[0, i].axis('off')
        output = np.clip(outputs[i], 0, 1)
        axes[1, i].imshow(output)
        axes[1, i].set_title('Reconstructed')
        axes[1, i].axis('off')
    plt.show()


# Evaluate the model
evaluate_neg(model_rgb_to_neg, test_dataloader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# 1. Extract features using the trained autoencoder
encoder = model_rgb_to_neg.encoder.eval()  # Set to evaluation mode

# 2. Define the MLP classifier
class LABToNEGMLPClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LABToNEGMLPClassifier, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        #x = x.view(-1, self.input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

writer = SummaryWriter()


rgb_to_neg_mlp = LABToNEGMLPClassifier(input_size=100352, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rgb_to_neg_mlp.parameters(), lr=0.001)
label_to_index = {'ants': 0, 'bees': 1}

# 5. Train the MLP classifier
num_epochs = 10
for epoch in range(num_epochs):
    rgb_to_neg_mlp.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_dataloader:
        # Forward pass
        encoder_output = encoder(inputs)

        # Flatten the output
        encoder_output_flat = encoder_output.view(encoder_output.size(0), -1)

        outputs = rgb_to_neg_mlp(encoder_output_flat)
        labels = [label_to_index[label] for label in labels]
        labels = torch.tensor(labels)


        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track running loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
    # Log metrics using Tensorboard
    writer.add_scalar('Loss/train',epoch_loss , epoch)
    writer.add_scalar('Accuracy/train',epoch_accuracy , epoch)

    # Validate the model
    rgb_to_neg_mlp.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            labels = [label_to_index[label] for label in labels]
            labels = torch.tensor(labels)

             # Forward pass
            encoder_output = encoder(inputs)

             # Flatten the output
            encoder_output_flat = encoder_output.view(encoder_output.size(0), -1)

            outputs = rgb_to_neg_mlp(encoder_output_flat)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Log validation accuracy
    writer.add_scalar('Accuracy/val', 100 * correct / total, epoch)

# 6. Use Tensorboard for visualization
# Make sure to close the Tensorboard writer after training
writer.close()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.datasets import fetch_openml
import numpy as np


# Define the CAE architecture
class CAERGBTOFLIP(nn.Module):
    def __init__(self):
        super(CAERGBTOFLIP, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(20),
            nn.LeakyReLU(0.2),
            nn.Conv2d(20, 40, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(40),
            nn.LeakyReLU(0.2)
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(40, 20, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.ConvTranspose2d(20, 3, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def horizontal_flip(images):
    # Flip images horizontally
    flipped_images = torch.flip(images, dims=[3])  # Flip along the width dimension (index 3)
    return flipped_images

# Initialize CAE model
model_rgb_to_flip = CAERGBTOFLIP()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_rgb_to_flip.parameters(), lr=0.001)

# Train the CAE model
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0

    for images,_ in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        images_flip = horizontal_flip(images)

        # Forward pass
        outputs = model_rgb_to_flip(images)

        # Compute the loss
        loss = criterion(outputs, images_flip)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)


    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f},')


In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define evaluation function
def evaluate_flip(model, dataloader):
    model.eval()
    with torch.no_grad():
        for images, _ in dataloader:
            outputs_flip = model(images)
            images_flip = horizontal_flip(images)
            images_flip = images_flip.permute(0, 2, 3, 1).numpy()

            images = images.permute(0, 2, 3, 1).numpy()
            outputs_flip = outputs_flip.permute(0, 2, 3, 1).numpy()

            visualize_flip(images, outputs_flip)

            outputs_flip_flat = outputs_flip.reshape(-1, 3)  # Reshape to 2D array
            images_flip_flat = images_flip.reshape(-1, 3)  # Reshape to 2D array

            mse_value = mse(images_flip_flat,outputs_flip_flat)


            ssim_value = ssim(images, outputs_flip, win_size=3, data_range=1, multichannel=True)
            psnr_value = psnr(images, outputs_flip, data_range=1)

            print(f'MSE: {mse_value:.4f}, SSIM: {ssim_value:.4f}, PSNR: {psnr_value:.4f}')

# Visualize input and reconstructed images
def visualize_flip(images, outputs):
    # Visualize the first few images and their reconstructions
    num_images = min(5, len(images))
    fig, axes = plt.subplots(2, num_images, figsize=(12, 4))
    for i in range(num_images):
        image = np.clip(images[i], 0, 1)
        axes[0, i].imshow(image)
        axes[0, i].set_title('Input')
        axes[0, i].axis('off')
        output = np.clip(outputs[i], 0, 1)
        axes[1, i].imshow(output)
        axes[1, i].set_title('Reconstructed')
        axes[1, i].axis('off')
    plt.show()


# Evaluate the model
evaluate_flip(model_rgb_to_flip, test_dataloader)